
## Dense Block architecture CIFAR dataset



In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_data = tf.data.Dataset.from_tensor_slices((train_images.astype(np.float32) / 255, train_labels.reshape(-1).astype(np.int32)))
train_data = train_data.shuffle(buffer_size=60000).batch(128).repeat()

test_data = tf.data.Dataset.from_tensor_slices((test_images.astype(np.float32) / 255, test_labels.reshape(-1).astype(np.int32))).batch(128)

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
tfkl = tf.keras.layers
inputs = tfkl.Input((32, 32, 3))

conv1 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv1")(inputs)

#dense block 1

conv11 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv11")(conv1)
concat11 = tfkl.concatenate([conv1, conv11])
pool11 = tfkl.MaxPool2D(3, 1, padding="same", name="pool11")(concat11)
conv12 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv12")(pool11)
concat12 = tfkl.concatenate([conv1, conv11, conv12])
pool12 = tfkl.MaxPool2D(3, 1, padding="same", name="pool12")(concat12)
conv13 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv13")(pool12)
concat13 = tfkl.concatenate([conv1, conv11, conv12, conv13])
pool13 = tfkl.MaxPool2D(3, 1, padding="same", name="pool13")(concat13)

conv2 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv2")(pool13)
pool2 = tfkl.MaxPool2D(3, 1, padding="same", name="pool2")(conv2)

#dense block 2
conv21 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv21")(pool2)
concat21 = tfkl.concatenate([conv2, conv21])
pool21 = tfkl.MaxPool2D(3, 1, padding="same", name="pool21")(concat21)
conv22 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv22")(pool21)
concat22 = tfkl.concatenate([conv2, conv21, conv22])
pool22 = tfkl.MaxPool2D(3, 1, padding="same", name="pool22")(concat22)
conv23 = tfkl.Conv2D(32, 3, activation=tf.nn.relu, padding="same",name="conv23")(pool22)
concat23 = tfkl.concatenate([conv2, conv21, conv22, conv23])
pool23 = tfkl.MaxPool2D(3, 1, padding="same", name="pool23")(concat23)

pool3 = tfkl.MaxPool2D(3, 1, padding="same", name="pool3")(pool23)


flatten = tf.keras.layers.Flatten()(pool3)
dense = tf.keras.layers.Dense(10, name="Dense")(flatten)

optimizer = tf.optimizers.Adam()

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = tf.metrics.SparseCategoricalAccuracy()




In [ ]:
model = tf.keras.Model(inputs, dense, name="Model")
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=train_acc_metric)
model.summary()
log = model.fit(train_data, steps_per_epoch=1000,epochs=2,validation_data=test_data)

Model: "Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1 (Conv2D)                 (None, 32, 32, 32)   896         ['input_1[0][0]']                
                                                                                                  
 conv11 (Conv2D)                (None, 32, 32, 32)   9248        ['conv1[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 32, 32, 64)   0           ['conv1[0][0]',                  
                                                                  'conv11[0][0]']             

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = model(images)
        xent = loss_fn(labels, logits)

    variables = model.trainable_variables
    gradients = tape.gradient(xent, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return xent, logits

In [ ]:

train_steps = 2000

start = time.time()
for step, (img_batch, lbl_batch) in enumerate(train_data):
    if step > train_steps:
        break

    xent, logits = train_step(img_batch, lbl_batch)

    if not step % 1000:
        train_acc_metric(lbl_batch, logits)
        acc = train_acc_metric.result()
        print("Loss: {} Accuracy: {}".format(xent, acc))
        train_acc_metric.reset_states()

        stop = time.time()
        print("took {} seconds\n".format(stop-start))
        start = time.time()

Loss: 0.8545383810997009 Accuracy: 0.7114928960800171
took 2.023367166519165 seconds

Loss: 0.6142902374267578 Accuracy: 0.8046875
took 134.46413278579712 seconds

Loss: 0.5070408582687378 Accuracy: 0.8203125
took 133.7143907546997 seconds

